In [ ]:
#from google.colab import drive
#drive.mount(r'C:\Users\vinee\Desktop\Mini Project 2B (ITM601)\awareness-of-waste-recycling-master')

In [1]:
%cd "C:\Users\vinee\Desktop\Mini Project 2B (ITM601)\awareness-of-waste-recycling-master"

C:\Users\vinee\Desktop\Mini Project 2B (ITM601)\awareness-of-waste-recycling-master


In [2]:
# !unzip recycling\ waste.zip

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# let's prepare the data and generate the data

from tensorflow.keras.preprocessing.image import ImageDataGenerator

gen_train = ImageDataGenerator(rescale = 1/255, shear_range = 0.2, zoom_range = 0.2, 
                               brightness_range = (0.1, 0.5), horizontal_flip=True)

train_data = gen_train.flow_from_directory(r"C:\Users\vinee\Desktop\Mini Project 2B (ITM601)\awareness-of-waste-recycling-master\recycling waste",
                                           target_size = (224, 224), batch_size = 32, class_mode="categorical")


Found 8369 images belonging to 9 classes.


In [5]:
# let's create a model
# here i'm going to use VGG16 model's parameter to solve this problem

from tensorflow.keras.applications.vgg16 import VGG16

# here i'm going to take input shape, weights and bias from imagenet and include top False means
# i want to add input, flatten and output layer by my self

vgg16 = VGG16(input_shape = (224, 224, 3), weights = "imagenet", include_top = False)

In [6]:
# now vgg16 weights are already train so i don't want to train that weights again
# so let's make trainable = False

for layer in vgg16.layers:
  layer.trainable = False

In [7]:
# let's add flatten layer or let's connect VGG16 with our own flatten layer

from tensorflow.keras import layers

x = layers.Flatten()(vgg16.output)

In [ ]:
# now let's add output layers or prediction layer

prediction = layers.Dense(units = 9, activation="softmax")(x)

# creating a model object

model = tf.keras.models.Model(inputs = vgg16.input, outputs=prediction)
model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics =["accuracy"])

result = model.fit_generator(train_data, epochs = 28, steps_per_epoch=len(train_data))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

C:\Users\vinee\AppData\Local\Temp/ipykernel_32764/2812959610.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  result = model.fit_generator(train_data, epochs = 28, steps_per_epoch=len(train_data))


Epoch 1/28
262/262 [==============================] - 916s 3s/step - loss: 1.9028 - accuracy: 0.3447
Epoch 2/28
262/262 [==============================] - 1110s 4s/step - loss: 1.5649 - accuracy: 0.4622
Epoch 3/28
262/262 [==============================] - 22734s 87s/step - loss: 1.4895 - accuracy: 0.4943
Epoch 4/28
262/262 [==============================] - 994s 4s/step - loss: 1.4012 - accuracy: 0.5160
Epoch 5/28
262/262 [==============================] - 1136s 4s/step - loss: 1.3587 - accuracy: 0.5327
Epoch 6/28
262/262 [==============================] - 1163s 4s/step - loss: 1.2618 - accuracy: 0.5695
Epoch 7/28
262/262 [==============================] - 1596s 6s/step - loss: 1.2710 - accuracy: 0.5737
Epoch 8/28
262/262 [==============================] - 1174s 4s/step - loss: 1.2585 - accuracy: 0.5718
Epoch 9/28
262/262 [==============================] - 999s 4s/step - loss: 1.1386 - accuracy: 0.6131
Epoch 10/28
262/262 [==============================] - 955s 4s/step - loss: 1.1459 

In [ ]:
# now let's compile the model

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics =["accuracy"])

result = model.fit_generator(train_data, epochs = 28, steps_per_epoch=len(train_data))

In [ ]:
from tensorflow.keras.preprocessing import image
output_class = ["batteries", "clothes", "e-waste", "glass", "light blubs", "metal", "organic", "paper", "plastic"]
def waste_prediction(new_image):
  test_image = image.load_img(new_image, target_size = (224,224))
  plt.axis("off")
  plt.imshow(test_image)
  plt.show()
 
  test_image = image.img_to_array(test_image) / 255
  test_image = np.expand_dims(test_image, axis=0)

  predicted_array = model.predict(test_image)
  predicted_value = output_class[np.argmax(predicted_array)]
  predicted_accuracy = round(np.max(predicted_array) * 100, 2)

  print("Your waste material is ", predicted_value, " with ", predicted_accuracy, " % accuracy")


In [ ]:
waste_prediction("toy.jpg")

In [ ]:
plt.title("Accuracy")
plt.plot(result.history["accuracy"])
plt.show()

In [ ]:
plt.title("Loss")
plt.plot(result.history["loss"])
plt.show()

In [ ]:
model.save("classifyWaste.h5")